In [ ]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

!pip install optuna -q

import pickle
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import cross_val_score, train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 5.9 MB/s eta 0:00:00


# 1. 데이터 로드

In [ ]:
print("="*60)
print("📂 데이터 로드")
print("="*60)

LOAD_PATH = '/content/drive/MyDrive/preprocessed_data.pkl'

with open(LOAD_PATH, 'rb') as f:
    data_dict = pickle.load(f)

X_train = data_dict['X_train_clf']
X_val = data_dict['X_val_clf']
X_test = data_dict['X_test_clf']
y_train = data_dict['y_train_clf']
y_val = data_dict['y_val_clf']
y_test = data_dict['y_test_clf']

# Train + Val 합치기
X_train_full = pd.concat([X_train, X_val], axis=0).reset_index(drop=True)
y_train_full = pd.concat([y_train, y_val], axis=0).reset_index(drop=True)

print(f"✅ Train+Val: {X_train_full.shape}, Test: {X_test.shape}")

📂 데이터 로드
✅ Train+Val: (87026, 45), Test: (9670, 45)


# 2. XGBoost - 기본

In [ ]:
print("\n" + "="*60)
print("🚀 XGBoost (기본 버전)")
print("="*60)

xgb_base = XGBClassifier(
    n_estimators=200,
    max_depth=10,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=1,  # -1 → 1
    eval_metric='mlogloss'
)

xgb_base.fit(X_train_full, y_train_full)  # X_train → X_train_full
xgb_base_pred = xgb_base.predict(X_test)

base_acc = accuracy_score(y_test, xgb_base_pred)
base_f1 = f1_score(y_test, xgb_base_pred, average='macro')

print(f"Accuracy: {base_acc:.4f}")
print(f"F1 Score (macro): {base_f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, xgb_base_pred, target_names=['Bad', 'Standard', 'Good']))



🚀 XGBoost Classifier
Accuracy: 0.8176
F1 Score (macro): 0.8104

Classification Report:
              precision    recall  f1-score   support

         Bad       0.80      0.82      0.81      5497
    Standard       0.84      0.82      0.83     10301
        Good       0.78      0.79      0.79      3542

    accuracy                           0.82     19340
   macro avg       0.81      0.81      0.81     19340
weighted avg       0.82      0.82      0.82     19340



# 3. XGBoost - 최적화

## 3.1. 샘플링

In [ ]:

X_sample, _, y_sample, _ = train_test_split(
    X_train_full, y_train_full, train_size=0.3, random_state=42, stratify=y_train_full
)
print(f"✅ 튜닝용 샘플: {X_sample.shape}")

✅ 튜닝용 샘플: (26107, 45)


## 3.2. Optuna 하이퍼파리미터 튜닝

In [ ]:
print("\n" + "="*60)
print("🚀 XGBoost Optuna 튜닝")
print("="*60)

def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [100, 200, 300]),
        'max_depth': trial.suggest_categorical('max_depth', [5, 7, 10]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.05, 0.1, 0.2]),
        'subsample': trial.suggest_categorical('subsample', [0.7, 0.8, 0.9]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.7, 0.8, 0.9]),
        'reg_alpha': trial.suggest_categorical('reg_alpha', [0, 0.1, 1]),
        'reg_lambda': trial.suggest_categorical('reg_lambda', [1, 2, 5]),
    }

    model = XGBClassifier(**params, random_state=42, eval_metric='mlogloss', n_jobs=1)
    scores = cross_val_score(model, X_sample, y_sample, cv=2, scoring='f1_macro', n_jobs=1)
    return scores.mean()

optuna.logging.set_verbosity(optuna.logging.WARNING)
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=15, show_progress_bar=True)

print(f"\n✅ 최적 파라미터: {study_xgb.best_params}")
print(f"✅ 최고 CV F1: {study_xgb.best_value:.4f}")


🚀 XGBoost Optuna 튜닝


  0%|          | 0/15 [00:00<?, ?it/s]


✅ 최적 파라미터: {'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.1, 'subsample': 0.7, 'colsample_bytree': 0.7, 'reg_alpha': 0, 'reg_lambda': 1}
✅ 최고 CV F1: 0.7086


## 3.3. XGBoost 학습 및 평가
* 하이퍼파라미터 튜닝

In [ ]:
# =============================================================================
# 최적 모델로 전체 데이터 학습
# =============================================================================
print("\n" + "="*60)
print("🎯 최종 XGBoost 학습")
print("="*60)

best_xgb = XGBClassifier(**study_xgb.best_params, random_state=42, eval_metric='mlogloss', n_jobs=1)
best_xgb.fit(X_train_full, y_train_full)
xgb_pred = best_xgb.predict(X_test)

print(f"\n📊 Test 성능:")
print(f"   Accuracy: {accuracy_score(y_test, xgb_pred):.4f}")
print(f"   F1 Score (macro): {f1_score(y_test, xgb_pred, average='macro'):.4f}")
print("\n" + classification_report(y_test, xgb_pred, target_names=['Bad', 'Standard', 'Good']))


🎯 최종 XGBoost 학습

📊 Test 성능:
   Accuracy: 0.8296
   F1 Score (macro): 0.8239

              precision    recall  f1-score   support

         Bad       0.81      0.83      0.82      2749
    Standard       0.85      0.83      0.84      5150
        Good       0.80      0.82      0.81      1771

    accuracy                           0.83      9670
   macro avg       0.82      0.83      0.82      9670
weighted avg       0.83      0.83      0.83      9670



In [ ]:
# =============================================================================
# 모델 저장
# =============================================================================
SAVE_PATH = '/content/drive/MyDrive/best_xgb_model.pkl'

save_dict = {
    'model': best_xgb,
    'best_params': study_xgb.best_params
}

with open(SAVE_PATH, 'wb') as f:
    pickle.dump(save_dict, f)

print(f"\n✅ 모델 저장 완료: {SAVE_PATH}")


✅ 모델 저장 완료: /content/drive/MyDrive/best_xgb_model.pkl


# 4. XGBoost - 최적화
* 피처 엔지니어링 + 선택
* 하이퍼파리미터 튜닝

## 4.1. 데이터 전처리

In [ ]:
# Train + Val 합치기
X_train_full = pd.concat([X_train, X_val], axis=0).reset_index(drop=True)
y_train_full = pd.concat([y_train, y_val], axis=0).reset_index(drop=True)

print(f"✅ Train+Val: {X_train_full.shape}, Test: {X_test.shape}")

✅ Train+Val: (87026, 45), Test: (9670, 45)


In [ ]:
# =============================================================================
# 피처 엔지니어링
# =============================================================================
print("\n" + "="*60)
print("📌 피처 엔지니어링")
print("="*60)

def create_features(df):
    data = df.copy()

    if 'outstanding_debt' in data.columns and 'annual_income' in data.columns:
        data['debt_to_income'] = data['outstanding_debt'] / (data['annual_income'] + 1)
    if 'total_emi_per_month' in data.columns and 'monthly_inhand_salary' in data.columns:
        data['emi_to_income'] = data['total_emi_per_month'] / (data['monthly_inhand_salary'] + 1)
    if 'monthly_balance' in data.columns and 'monthly_inhand_salary' in data.columns:
        data['balance_to_income'] = data['monthly_balance'] / (data['monthly_inhand_salary'] + 1)
    if 'amount_invested_monthly' in data.columns and 'monthly_inhand_salary' in data.columns:
        data['invest_to_income'] = data['amount_invested_monthly'] / (data['monthly_inhand_salary'] + 1)
    if 'num_credit_card' in data.columns and 'credit_utilization_ratio' in data.columns:
        data['credit_load'] = data['num_credit_card'] * data['credit_utilization_ratio']
    if 'delay_from_due_date' in data.columns and 'num_of_delayed_payment' in data.columns:
        data['delay_severity'] = data['delay_from_due_date'] * data['num_of_delayed_payment']
    if 'num_of_loan' in data.columns and 'num_bank_accounts' in data.columns:
        data['loan_per_account'] = data['num_of_loan'] / (data['num_bank_accounts'] + 1)
    if 'num_credit_inquiries' in data.columns and 'credit_history_age' in data.columns:
        data['inquiries_per_year'] = data['num_credit_inquiries'] / (data['credit_history_age'] / 12 + 1)
    if 'num_bank_accounts' in data.columns and 'credit_history_age' in data.columns:
        data['accounts_per_year'] = data['num_bank_accounts'] / (data['credit_history_age'] / 12 + 1)
    if 'total_emi_per_month' in data.columns and 'outstanding_debt' in data.columns and 'interest_rate' in data.columns:
        data['payment_burden'] = data['total_emi_per_month'] + data['outstanding_debt'] * (data['interest_rate'] / 100)
    if 'annual_income' in data.columns:
        data['log_income'] = np.log1p(data['annual_income'])
    if 'outstanding_debt' in data.columns:
        data['log_debt'] = np.log1p(data['outstanding_debt'])

    return data

X_train_fe = create_features(X_train_full)
X_test_fe = create_features(X_test)

print(f"✅ 피처: {X_train_full.shape[1]}개 → {X_train_fe.shape[1]}개")


📌 피처 엔지니어링
✅ 피처: 45개 → 56개


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# =============================================================================
# Feature Selection
# =============================================================================
print("\n" + "="*60)
print("📌 Feature Selection")
print("="*60)

rf_temp = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=1)
rf_temp.fit(X_train_fe, y_train_full)

importance_df = pd.DataFrame({
    'feature': X_train_fe.columns,
    'importance': rf_temp.feature_importances_
}).sort_values('importance', ascending=False)

top_features = importance_df.head(20)['feature'].tolist()
X_train_selected = X_train_fe[top_features]
X_test_selected = X_test_fe[top_features]

print(f"✅ {len(top_features)}개 피처 선택: {top_features[:5]}...")


📌 Feature Selection
✅ 20개 피처 선택: ['credit_mix', 'log_debt', 'outstanding_debt', 'interest_rate', 'payment_burden']...


In [ ]:

# =============================================================================
# 기본 XGBoost
# =============================================================================
print("\n" + "="*60)
print("🚀 XGBoost (기본 버전)")
print("="*60)

xgb_base = XGBClassifier(
    n_estimators=200,
    max_depth=10,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=1,
    eval_metric='mlogloss'
)

xgb_base.fit(X_train_selected, y_train_full)
xgb_base_pred = xgb_base.predict(X_test_selected)

base_acc = accuracy_score(y_test, xgb_base_pred)
base_f1 = f1_score(y_test, xgb_base_pred, average='macro')

print(f"Accuracy: {base_acc:.4f}")
print(f"F1 Score (macro): {base_f1:.4f}")


🚀 XGBoost (기본 버전)
Accuracy: 0.8133
F1 Score (macro): 0.8062


## 4.2. Optuna - 하이퍼파리미터 튜닝

In [ ]:
# =============================================================================
# 샘플링
# =============================================================================
X_sample, _, y_sample, _ = train_test_split(
    X_train_selected, y_train_full,  # ✅ 수정됨!
    train_size=0.3, random_state=42, stratify=y_train_full
)
print(f"✅ 튜닝용 샘플: {X_sample.shape}")

✅ 튜닝용 샘플: (26107, 20)


In [ ]:
print("\n" + "="*60)
print("🚀 XGBoost (Optuna 최적화)")
print("="*60)

def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [100, 200, 300]),
        'max_depth': trial.suggest_categorical('max_depth', [5, 7, 10]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.05, 0.1, 0.2]),
        'subsample': trial.suggest_categorical('subsample', [0.7, 0.8, 0.9]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.7, 0.8, 0.9]),
        'reg_alpha': trial.suggest_categorical('reg_alpha', [0, 0.1, 1]),
        'reg_lambda': trial.suggest_categorical('reg_lambda', [1, 2, 5]),
    }

    model = XGBClassifier(**params, random_state=42, eval_metric='mlogloss', n_jobs=1)
    scores = cross_val_score(model, X_sample, y_sample, cv=2, scoring='f1_macro', n_jobs=1)
    return scores.mean()

optuna.logging.set_verbosity(optuna.logging.WARNING)
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=15, show_progress_bar=True)

print(f"\n✅ 최적 파라미터: {study_xgb.best_params}")

xgb_opt = XGBClassifier(**study_xgb.best_params, random_state=42, eval_metric='mlogloss', n_jobs=1)
xgb_opt.fit(X_train_selected, y_train_full)
xgb_opt_pred = xgb_opt.predict(X_test_selected)

opt_acc = accuracy_score(y_test, xgb_opt_pred)
opt_f1 = f1_score(y_test, xgb_opt_pred, average='macro')

print(f"Accuracy: {opt_acc:.4f}")
print(f"F1 Score (macro): {opt_f1:.4f}")


🚀 XGBoost (Optuna 최적화)


  0%|          | 0/15 [00:00<?, ?it/s]


✅ 최적 파라미터: {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.05, 'subsample': 0.9, 'colsample_bytree': 0.8, 'reg_alpha': 0, 'reg_lambda': 2}
Accuracy: 0.8041
F1 Score (macro): 0.7954


## 4.3. XGBoost 학습 및 성능 평가

In [ ]:
# =============================================================================
# 비교 결과
# =============================================================================
print("\n" + "="*60)
print("📊 기본 vs 최적화 비교")
print("="*60)

comparison = pd.DataFrame({
    'Model': ['XGBoost (기본)', 'XGBoost (Optuna)'],
    'Accuracy': [base_acc, opt_acc],
    'F1 (macro)': [base_f1, opt_f1]
})
print(comparison.to_string(index=False))


📊 기본 vs 최적화 비교
           Model  Accuracy  F1 (macro)
    XGBoost (기본)  0.813340    0.806182
XGBoost (Optuna)  0.804137    0.795399


In [ ]:
# =============================================================================
# 모델 저장 (더 좋은 모델 + top_features 함께 저장)
# =============================================================================
SAVE_PATH = '/content/drive/MyDrive/best_xgb_model_2.pkl'

best_model = xgb_opt if opt_f1 > base_f1 else xgb_base
best_params = study_xgb.best_params if opt_f1 > base_f1 else {'default': True}

save_dict = {
    'model': best_model,
    'best_params': best_params,
    'top_features': top_features  # ✅ 앙상블용 피처 목록 저장!
}

with open(SAVE_PATH, 'wb') as f:
    pickle.dump(save_dict, f)

print(f"\n✅ 모델 저장 완료: {SAVE_PATH}")
print(f"   저장된 피처 수: {len(top_features)}개")


✅ 모델 저장 완료: /content/drive/MyDrive/best_xgb_model_2.pkl
   저장된 피처 수: 20개
